# Statistische Analyse der Vergleichbarkeit von ICF-Körben, die über WHODAS und Coresets befüllt werden

In [1]:
import pandas as pd
from collections import Counter
import itertools

Einlesen der WHODAS-ICF und Contextfaktoren ICFs (=Env), Erzeugen eines Dataframes mit den Spalten "whodas" und "env"

In [51]:
whodas=pd.read_json('whodas12_de.json')
env=pd.read_json('env_factors_de.json')
df=pd.DataFrame(columns=['whodas'])
df['whodas']=list(itertools.chain.from_iterable( [x.split(',') for x in whodas.loc['l']]))
elist=list(itertools.chain.from_iterable( [x.split(',') for x in env.loc['l']]))
df=pd.concat([df,pd.Series(elist,name='env').to_frame()],axis=1)
df=df[df['whodas']!='']
df=df[df['env']!='']

Einlesen der Coreset-ICFs. Zusammenführen der Dataframes

In [63]:
coresets = pd.read_json('coresets.json')
a=[]
for c in coresets.columns:
    l=coresets.loc['items',c]
    a.append(pd.Series(l,name='coreset_'+c))
    
df2=pd.concat(a, axis='columns',ignore_index=False)

In [64]:
df3=pd.concat([df,df2],axis='columns',ignore_index=False)

In [65]:
df4_whodas=df3.applymap(lambda x: x in df3['whodas'].values)
df4_env=df3.applymap(lambda x: x in df3['env'].values)

## Zahl der gültigen Datensätze
Anzeigen der ICF-Items pro Korb

In [66]:
df3.count()

whodas                     40
env                        35
coreset_generic             7
coreset_rehabilitation     30
coreset_minimal-context    12
coreset_L-Spine            35
coreset_Osteoarthritis     13
coreset_musc-acute         28
coreset_musc-postacute     31
coreset_chronic-pain       24
coreset_osteoporosis       12
coreset_rheuma             20
coreset_khk                19
coreset_diabetes           27
coreset_adipositas          8
coreset_copd               14
coreset_neuro-postacute    38
coreset_depression         31
coreset_work-reha          13
coreset_geriatrie          38
dtype: int64

## Überdeckung von Coreset-ICF-Items mit Whodas und Env Korb
Angezeigt wird der Anteil von Fragen aus Whodas oder Env, die auch im Coreset enthalten sind.

In [67]:
df4_whodas.apply(lambda x: sum(x)/40)

whodas                     1.000
env                        0.000
coreset_generic            0.075
coreset_rehabilitation     0.300
coreset_minimal-context    0.000
coreset_L-Spine            0.175
coreset_Osteoarthritis     0.050
coreset_musc-acute         0.125
coreset_musc-postacute     0.175
coreset_chronic-pain       0.150
coreset_osteoporosis       0.050
coreset_rheuma             0.050
coreset_khk                0.125
coreset_diabetes           0.025
coreset_adipositas         0.050
coreset_copd               0.075
coreset_neuro-postacute    0.175
coreset_depression         0.125
coreset_work-reha          0.125
coreset_geriatrie          0.175
dtype: float64

In [68]:
df4_env.apply(lambda x: sum(x)/35)

whodas                     0.000000
env                        1.000000
coreset_generic            0.000000
coreset_rehabilitation     0.000000
coreset_minimal-context    0.257143
coreset_L-Spine            0.142857
coreset_Osteoarthritis     0.114286
coreset_musc-acute         0.028571
coreset_musc-postacute     0.114286
coreset_chronic-pain       0.057143
coreset_osteoporosis       0.057143
coreset_rheuma             0.114286
coreset_khk                0.000000
coreset_diabetes           0.171429
coreset_adipositas         0.057143
coreset_copd               0.114286
coreset_neuro-postacute    0.114286
coreset_depression         0.114286
coreset_work-reha          0.114286
coreset_geriatrie          0.114286
dtype: float64

## Verteilungsmatrix

In [76]:
all_icf_items=list(itertools.chain.from_iterable( [df3[s].dropna().to_list() for s in df3]))

In [82]:
# Zahl aller verwendeten ICF Items in allen Körben
cnt = Counter()
for text in all_icf_items:
    cnt[text] += 1
    
len(cnt)

173

In [89]:
# die 10 häufigsten
commons = cnt.most_common(10)

In [90]:
commons

[('d450', 15),
 ('e110', 11),
 ('e355', 11),
 ('e310', 10),
 ('b455', 10),
 ('d240', 10),
 ('e580', 9),
 ('b130', 9),
 ('b280', 9),
 ('d850', 8)]

In [91]:
# Erzeugen der Häufigkeitsmatrix als Dataframe
df_weights=df3.applymap(lambda x: cnt[x])
df_weights.style.background_gradient(cmap='Blues')

,whodas,env,coreset_generic,coreset_rehabilitation,coreset_minimal-context,coreset_L-Spine,coreset_Osteoarthritis,coreset_musc-acute,coreset_musc-postacute,coreset_chronic-pain,coreset_osteoporosis,coreset_rheuma,coreset_khk,coreset_diabetes,coreset_adipositas,coreset_copd,coreset_neuro-postacute,coreset_depression,coreset_work-reha,coreset_geriatrie
0,6,1,9,9,11,9,9,9,5,9,7,9,9,9,9,2,1,1,9,5
1,2,1,7,5,8,5,1,7,1,5,9,10,7,2,4,2,1,1,2,2
2,1,1,9,7,4,7,7,3,9,2,5,5,9,1,10,10,1,1,10,10
3,5,4,8,9,2,9,4,2,2,7,7,7,2,2,15,3,2,1,4,3
4,1,1,15,10,3,10,5,10,4,1,4,2,3,3,6,8,1,1,10,5
5,2,1,6,5,3,5,3,1,5,9,15,8,3,3,6,15,2,1,2,1
6,4,1,8,1,4,1,3,5,7,10,4,7,10,10,11,6,3,2,4,1
7,2,2,0,5,10,7,15,5,4,7,11,2,3,4,10,5,2,1,8,8
8,5,1,0,7,3,2,6,2,1,1,11,3,4,1,0,11,1,1,2,1
9,1,1,0,8,11,4,8,10,2,5,9,15,8,1,0,8,4,1,10,7


In [88]:
# Normalisieren der Gewichte auf die Korbgröße (Column). Zahlen über 2 besagen, dass im Durchschnitt, jedes Item nochmal in einem anderen Korb zu finden ist.
df_weights.apply(lambda x: sum(x)/len(x))

whodas                     2.976744
env                        2.186047
coreset_generic            1.441860
coreset_rehabilitation     4.209302
coreset_minimal-context    1.697674
coreset_L-Spine            4.790698
coreset_Osteoarthritis     1.930233
coreset_musc-acute         3.581395
coreset_musc-postacute     4.116279
coreset_chronic-pain       3.534884
coreset_osteoporosis       2.093023
coreset_rheuma             3.093023
coreset_khk                2.790698
coreset_diabetes           3.093023
coreset_adipositas         1.651163
coreset_copd               1.930233
coreset_neuro-postacute    3.488372
coreset_depression         2.488372
coreset_work-reha          1.744186
coreset_geriatrie          3.976744
dtype: float64

# Monte Carlo Simulation
Aufgabe: Jeder Spieler zieht 5 (3,6,7...) ICF Items aus dem Korb. Mit welcher Wahrscheinlichkeit überdeckt sich die Auswahl aus Whodas oder Env mit einer Ziehung aus den anderen Körben mindestens zu 50% (70,80,90%..) ?